In [ ]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Define a retriever from vector database
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

retriever = FAISS.load_local(
    folder_path="training-data",
    index_name="throwing-exceptions",
    embeddings=OpenAIEmbeddings()
).as_retriever()

In [ ]:
retriever.get_relevant_documents("Who is Sowren Sen?")

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    retriever=retriever, 
    name="sowren_sen_and_throwing_exceptions",
    description="Searches and returns documents regarding Sowren Sen and his blog Throwing Exceptions"
)

tools = [tool]

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=10,
    return_messages=True,
)

In [ ]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

template="""
You are a helpful assistant of Sowren Sen's blog 'Throwing Exceptions'. People will ask questions about Sowren, his contact, articles he written, how many tags or categories he have in his blog etc. You will be helping them to provide replies to their queries based on the given information from his blog. If some information is not found, politely tell the visitors that you don't have this information at this moment right now and they can always reach out to Sowren using his contact. Also provide them the contacts. Beside this, you can remind them to use the search functionality which can be accessed through the 🔍 icon on the top navigation bar or simply pressing "/" on their keyboard. Stick to the information you have, do not generate anything by yourself that does not exist. 
"""

system_message = SystemMessage(content=template)

prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name='chat_history')]
)

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [ ]:
from langchain.agents import AgentExecutor

ae = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=conversational_memory,
    verbose=True,
    return_intermediate_steps=False
)

In [ ]:
ae({"input": "Who is Sowren Sen?"})["output"]

In [ ]:
ae({"input": "How can I reach out to him?"})["output"]

In [ ]:
ae({"input": "Does he have any github links?"})["output"]

In [ ]:
ae({"input": "Is that the only link?"})["output"]

In [ ]:
ae({"input": "Give me list of his open-source projects in bullet point"})["output"]

In [ ]:
ae({"input": "give me a list of his laravel articles"})["output"]